# Fine-tune Qwen3:8b for Dino Assistant

In [ ]:
# Install dependencies
!pip install -q transformers accelerate peft datasets

In [ ]:
import json, os
from google.colab import userdata

# Clone training data
!git clone https://github.com/RekitRex21/Dino-Training.git
os.chdir('Dino-Training')

In [ ]:
# Load and combine training data
data = []

# Load JSON files
for f in ['security_analysis.json', 'trading.json']:
    with open(f) as fp:
        data.extend(json.load(fp))

# Load JSONL
with open('rex_assistant.jsonl') as fp:
    for line in fp:
        data.append(json.loads(line))

print(f"Loaded {len(data)} training examples")

In [ ]:
# Format for training
formatted_data = []
for item in data:
    text = f"Instruction: {item['instruction']}\n\nInput: {item.get('input', '')}\n\nOutput: {item['output']}"
    formatted_data.append({'text': text})

print(f"Formatted {len(formatted_data)} examples")

In [ ]:
# Load tokenizer - USE QWEN
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct", trust_remote_code=True)
print("Tokenizer loaded")

In [ ]:
# Tokenize
def tokenize(sample):
    return tokenizer(sample['text'], truncation=True, max_length=2048)

from datasets import Dataset
ds = Dataset.from_list(formatted_data)
ds = ds.map(tokenize, batched=True)
print(f"Tokenized dataset: {len(ds)} examples")

In [ ]:
# Load model with 4-bit quantization
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
print("Model loaded!")

In [ ]:
# Setup LoRA
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# Train!
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="./qwen3-dino",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=50,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds,
)

trainer.train()

In [ ]:
# Save model
model.save_pretrained("./qwen3-dino-final")
tokenizer.save_pretrained("./qwen3-dino-final")
print("Model saved!")

In [ ]:
# Download for local use
!zip -r qwen3-dino-model.zip qwen3-dino-final/
print("Zipped! Right-click to download.")